In [38]:
# ── 1. Project paths ──────────────────────────────────────────────────────
from pathlib import Path
import pandas as pd

ROOT = Path.cwd()
while not (ROOT / ".gitignore").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

CLEAN_DIR = ROOT / "data" / "clean"
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

RUNS_PATH           = CLEAN_DIR / "runs_cleaned.csv"
PARCELS_LATEST_PATH = CLEAN_DIR / "parcels_jefferson_latest.csv"
PARCELS_PANEL_PATH  = CLEAN_DIR / "parcels_jefferson_panel.csv"
OUTPUT_PATH         = CLEAN_DIR / "street_year_panel.csv"

In [39]:
# ── 2. Load Data ───────────────────────────────────────────────────────────
runs           = pd.read_csv(RUNS_PATH)
parcels_latest = pd.read_csv(PARCELS_LATEST_PATH)
parcels_panel  = pd.read_csv(PARCELS_PANEL_PATH)

In [40]:
# ── 3. Parse Dates in Runs ────────────────────────────────────────────────
runs['date']    = pd.to_datetime(runs['date'])
runs['year']    = runs['date'].dt.year
runs['weekday'] = runs['date'].dt.day_name()

In [41]:
# ── 4. Impute Missing Categories in Runs ──────────────────────────────────
for col in ['prop_use_desc', 'aid_desc', 'inc_type_desc']:
    runs[col] = runs[col].fillna('Unknown')

In [42]:
# ── 5. Feature Engineering on Parcels Latest ──────────────────────────────
parcels_latest['prop_age'] = 2025 - parcels_latest['yearblt']
valid_area = parcels_latest['stories_x_sqft'] > 0
parcels_latest.loc[valid_area, 'value_per_sqft'] = (
    parcels_latest.loc[valid_area, 'apprtot'] /
    parcels_latest.loc[valid_area, 'stories_x_sqft']
)
parcels_latest.loc[~valid_area, 'value_per_sqft'] = pd.NA

In [43]:
# ── 6. Compute Panel Growth Rates ─────────────────────────────────────────
parcels_panel = parcels_panel.sort_values(['parcel_id', 'source_year'])
parcels_panel['appr_growth'] = (
    parcels_panel
      .groupby('parcel_id')['apprtot']
      .pct_change()
)
parcels_panel['sqft_growth'] = (
    parcels_panel
      .groupby('parcel_id')['stories_x_sqft']
      .pct_change()
)

C:\Users\JosephWhite\AppData\Local\Temp\ipykernel_17120\2310508690.py:11: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()


In [47]:
# ── 7. Extract street names (drop house numbers) ─────────────────────────
# From runs: use 'addr_norm'
runs['street_name'] = runs['addr_norm'].str.replace(r'^\d+\s+', '', regex=True)

# From parcels: use 'staddr_norm'
parcels_latest['street_name'] = parcels_latest['staddr_norm'].str.replace(r'^\d+\s+', '', regex=True)

In [48]:
# ── 8. Merge Runs + Parcels on street_name ────────────────────────────────
merged = runs.merge(
    parcels_latest,
    on='street_name',
    how='left',
    suffixes=('_run', '_parcel')
)

In [50]:
# ── 9. Aggregate to Street‐Year Panel ─────────────────────────────────────
# a) Total runs per street-year
runs_by_street = (
    merged
      .groupby(['street_name','year'])
      .size()
      .rename('run_count')
)

# b) Total building area per street-year
area_by_street = (
    merged
      .drop_duplicates(subset=['street_name','parcel_id','year'])
      .groupby(['street_name','year'])['stories_x_sqft']
      .sum()
      .rename('total_sqft')
)

# c) Street-level average growth metrics
growth_with_addr = (
    parcels_panel
      .merge(
          parcels_latest[['parcel_id','source_year','street_name']],
          left_on=['parcel_id','source_year'],
          right_on=['parcel_id','source_year'],
          how='inner'
      )
)
growth_by_street = (
    growth_with_addr
      .groupby(['street_name','source_year'])
      .agg({
        'appr_growth':'mean',
        'sqft_growth':'mean'
      })
      .rename_axis(index={'source_year':'year'})
)

# d) Combine into one street×year table
street_year_panel = pd.concat(
    [runs_by_street, area_by_street, growth_by_street],
    axis=1
).reset_index()

In [51]:
# ── 10. Save Panel ─────────────────────────────────────────────────────────
street_year_panel.to_csv(OUTPUT_PATH, index=False)